In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Intake the training data and only keep the necessary columns

In [ ]:
data = pd.read_csv('Twitter_training.csv', names=["Tweet_ID", "Entity", "Sentiment", "Text"])
data = data[['Text','Sentiment']]
data.head()

Goal is to identify Positive and Negative tweets, drop everything else and keep only valid text

In [ ]:
import re
data = data[data.Sentiment != "Neutral"]
data = data[data.Sentiment != "Irrelevant"]
data.Text = data.Text.apply(lambda x: str(x).lower())
data.Text = data.Text.apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

Lemmatize words to elimate stopwords that provide no context

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatiser = WordNetLemmatizer()
stopwords = set(stopwords.words())
def remove_stopwords(ls):
    # Removes stop words and lemmatises
    ls = [lemmatiser.lemmatize(word) for word in ls if word not in (stopwords) and (word.isalpha())]
    
    ls = " ".join(ls)
    return ls

data.Text = data.Text.apply(word_tokenize)
data.Text = data.Text.apply(remove_stopwords)

Tokenize the words to eliminate variations of words

In [ ]:
print(data[data.Sentiment == 'Positive'].size)
print(data[data.Sentiment == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_features = 1000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data.Text.values)
X = tokenizer.texts_to_sequences(data.Text.values)
X = pad_sequences(X)

Visualization of data after applying transformations

In [ ]:
data.head()

Here, we used LSTM, a recurrent neural network implmentation, to differentiate and distinguish the context of the content as the method to determining sentiment. 

The LSTM layer only uses the dropout and not the recurrent_dropout parameter in order to accelerate training. Recurrent_dropout is currently not supported by Nvidia CUDNN and will prevent the model from utilizing GPU acceleration.

The Dense layer should only be 2 units as our sentiment has only 2 possible values

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 128
lstm_out = 196

model = Sequential([
     Embedding(max_features, embed_dim, input_length = X.shape[1]),
     SpatialDropout1D(0.4),
     LSTM(lstm_out, dropout=0.2),
     Dense(2, activation='softmax')
])

model.compile(
     loss='categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)
print(model.summary())

Split the dataset into training and validation

In [ ]:
Y = pd.get_dummies(data.Sentiment).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='accuracy', patience=4)

model.fit(X_train, Y_train, epochs = 10, batch_size=32, callbacks=[callback], verbose = 1)

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

Run tests on validation set

In [ ]:
accuracy = {
    'pos_cnt':0, 
    'neg_cnt':0, 
    'pos_correct':0, 
    'neg_correct':0
}

def inc(count):
    count+=1

for x in range(len(X_validate)):
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]), batch_size=1)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        accuracy['neg_correct' if np.argmax(Y_validate[x]) == 0 else 'pos_correct'] +=1
       
    accuracy['neg_cnt' if np.argmax(Y_validate[x]) == 0 else 'pos_cnt'] +=1

print("pos_acc", accuracy['pos_correct']/accuracy['pos_cnt']*100, "%")
print("neg_acc", accuracy['neg_correct']/accuracy['neg_cnt']*100, "%")

Vectorize the tweet by the pre-fitted tokenizer instance then pad the tweet to have the same dimensions as the input

In [ ]:
def apply_prediction(twt):
    twtData = tokenizer.texts_to_sequences([twt])
    twtData = pad_sequences(twtData, maxlen=28, dtype='int32', value=0)
    print(twtData)
    sentiment = model.predict(twtData,batch_size=1,verbose = 2)[0]
    sentimentValue = "negative" if(np.argmax(sentiment) == 0) else "positive"
    return sentimentValue
    
twt = 'The new CoD is pretty lit'
print(apply_prediction(twt))